In [1]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms import OpenAI
from langchain.agents import AgentExecutor

In [2]:
davinci = OpenAI(model_name="text-davinci-003", temperature=0)

In [4]:
db = SQLDatabase.from_uri("sqlite:///spider/database/soccer_1/soccer_1.sqlite")
toolkit = SQLDatabaseToolkit(db=db)

agent_executor = create_sql_agent(
    llm=davinci,
    toolkit=toolkit,
    verbose=True
)

In [5]:
agent_executor.run("Describe this database and tell me what it stores.")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: ""
Observation: Team, Player, League, Team_Attributes, Player_Attributes, sqlite_sequence, Country
Thought: I can use the schema_sql_db tool to get more information about the tables.
Action: schema_sql_db
Action Input: "Team, Player, League, Team_Attributes, Player_Attributes, Country"
Observation: 
CREATE TABLE "Country" (
	id INTEGER, 
	name TEXT, 
	PRIMARY KEY (id)
)

SELECT * FROM 'Country' LIMIT 3;
id	name
1	Belgium
1729	England
4769	France


CREATE TABLE "Player" (
	id INTEGER, 
	player_api_id INTEGER, 
	player_name TEXT, 
	player_fifa_api_id INTEGER, 
	birthday TEXT, 
	height INTEGER, 
	weight INTEGER, 
	PRIMARY KEY (id)
)

SELECT * FROM 'Player' LIMIT 3;
id	player_api_id	player_name	player_fifa_api_id	birthday	height	weight
1	505942	Aaron Appindangoye	218353	1992-02-29 00:00:00	182.88	187
2	155782	Aaron Cresswell	189615	1989-12-15 00:00:00	170.18	146
3	162549	Aaron Doran	186170	1991-05-13 00:00:00	

'This database stores information about teams, players, leagues, and countries in a soccer database.'

In [15]:
agent_executor.run("What can you tell me about the players?")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: ""
Observation: Team, Player, League, Team_Attributes, Player_Attributes, sqlite_sequence, Country
Thought: I should look at the schema of the Player table to see what columns I can query.
Action: schema_sql_db
Action Input: "Player"
Observation: 
CREATE TABLE "Player" (
	id INTEGER, 
	player_api_id INTEGER, 
	player_name TEXT, 
	player_fifa_api_id INTEGER, 
	birthday TEXT, 
	height INTEGER, 
	weight INTEGER, 
	PRIMARY KEY (id)
)

SELECT * FROM 'Player' LIMIT 3;
id	player_api_id	player_name	player_fifa_api_id	birthday	height	weight
1	505942	Aaron Appindangoye	218353	1992-02-29 00:00:00	182.88	187
2	155782	Aaron Cresswell	189615	1989-12-15 00:00:00	170.18	146
3	162549	Aaron Doran	186170	1991-05-13 00:00:00	170.18	163
Thought: I should query the Player table for the relevant columns.
Action: query_sql_db
Action Input: SELECT player_name, birthday, height, weight FROM Player ORDER BY player_name LIMIT 10
Obse

'Here are the player names, birthdays, heights, and weights of the first 10 players in the database, ordered by player name: Aaron Appindangoye (1992-02-29, 182.88 cm, 187 kg), Aaron Cresswell (1989-12-15, 170.18 cm, 146 kg), Aaron Doran (1991-05-13, 170.18 cm, 163 kg), Aaron Galindo (1982-05-08, 182.88 cm, 198 kg), Aaron Hughes (1979-11-08, 182.88 cm, 154 kg), Aaron Hunt (1986-09-04, 182.88 cm, 161 kg), Aaron Kuhl (1996-01-30, 172.72 cm, 146 kg), Aaron Lennon (1987-04-16, 165.1 cm, 139 kg), Aaron Lennox (1993-02-19, 190.5 cm, 181 kg), Aaron Meijers (1987-10-28, 175.26 cm, 170 kg).'

In [16]:
agent_executor.run("What is the name of the player with the highest heading accuracy?")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: ""
Observation: Team, Player, League, Team_Attributes, Player_Attributes, sqlite_sequence, Country
Thought: I should look at the Player_Attributes table to see if it has the information I need.
Action: schema_sql_db
Action Input: "Player_Attributes"
Observation: 
CREATE TABLE "Player_Attributes" (
	id INTEGER, 
	player_fifa_api_id INTEGER, 
	player_api_id INTEGER, 
	date TEXT, 
	overall_rating INTEGER, 
	potential INTEGER, 
	preferred_foot TEXT, 
	attacking_work_rate TEXT, 
	defensive_work_rate TEXT, 
	crossing INTEGER, 
	finishing INTEGER, 
	heading_accuracy INTEGER, 
	short_passing INTEGER, 
	volleys INTEGER, 
	dribbling INTEGER, 
	curve INTEGER, 
	free_kick_accuracy INTEGER, 
	long_passing INTEGER, 
	ball_control INTEGER, 
	acceleration INTEGER, 
	sprint_speed INTEGER, 
	agility INTEGER, 
	reactions INTEGER, 
	balance INTEGER, 
	shot_power INTEGER, 
	jumping INTEGER, 
	stamina INTEGER, 
	strength INTEGE

'The player with the highest heading accuracy is player_api_id 38567 with a score of 98.'